<a href="https://colab.research.google.com/github/lessw2020/training-detr/blob/master/training_detr_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Example of how to setup and train your own custom dataset using DETR
# In this notebook, we'll do fine tuning of the existing head
# For larger datasets (say > 10K images) training from scratch may be preferred and we'll do a different notebook for that



In [0]:
# 

# Setup PyTorch environment and DETR

In [6]:
!pip install torch torchvision


In [11]:
import torch; 
torch_version = torch.__version__
assert float(torch_version[:3]) >= 1.40   # make sure 1.40 or higher
torch_version


'1.5.0+cu101'